This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.2_tutorials/wallaroo-model-operations-tutorials/deploy/by-framework/pytorch).

## Wallaroo Model Upload via the Wallaroo SDK: Pytorch Multiple Input Output

The following tutorial demonstrates how to upload a Pytorch Multiple Input Output model to a Wallaroo instance.

### Tutorial Goals

Demonstrate the following:

* Upload a Pytorch Multiple Input Output to a Wallaroo instance.
* Create a pipeline and add the model as a pipeline step.
* Perform a sample inference.

### Prerequisites

* Wallaroo Version 2023.2.1 or above instance.

### References

* [Wallaroo MLOps API Essentials Guide: Model Upload and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essentials-guide-model-uploads/)
* [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/)
* [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd


### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more details on logging in through Wallaroo, see the [Wallaroo SDK Essentials Guide: Client Connection](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client()

### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.



In [3]:
workspace_name = f'pytorch-multi-io'
pipeline_name = f'pytorch-multi-io'

model_name = 'pytorch-multi-io'
model_file_name = "./models/model-auto-conversion_pytorch_multi_io_model.pt"

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Configure Data Schemas

The following parameters are required for PyTorch models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a PyTorch model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Upload Method Optional, PyTorch model Required*) | Set as the `Framework.PyTorch`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, PyTorch model Required*) | The input schema in Apache Arrow schema format. Note that float values **must** be `pyarrow.float32()`. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, PyTorch model Required*) | The output schema in Apache Arrow schema format. Note that float values **must** be `pyarrow.float32()`. |
| `convert_wait` | `bool` (*Upload Method Optional, PyTorch model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

In [5]:
input_schema = pa.schema([
    pa.field('input_1', pa.list_(pa.float32(), list_size=10)),
    pa.field('input_2', pa.list_(pa.float32(), list_size=5))
])
output_schema = pa.schema([
    pa.field('output_1', pa.list_(pa.float32(), list_size=3)),
    pa.field('output_2', pa.list_(pa.float32(), list_size=2))
])

### Upload Model

The model will be uploaded with the framework set as `Framework.PYTORCH`.

In [6]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.PYTORCH, 
                        input_schema=input_schema, 
                        output_schema=output_schema
                       )
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime..
Ready


Name,pytorch-multi-io
Version,4ecb9de8-60fa-4a5f-b8fb-17e91a788cdb
File Name,model-auto-conversion_pytorch_multi_io_model.pt
SHA,792db9ee9f41aded3c1d4705f50ccdedd21cafb8b6232c03e4a849b6da1050a8
Status,ready
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2024-22-Jul 21:36:30
Workspace id,157
Workspace name,pytorch-multi-io


### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [8]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [9]:
# clear the pipeline if it was used before
pipeline.clear()

pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

Waiting for deployment - this will take up to 45s ........... ok


{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.4.2.28',
   'name': 'engine-84dddfb47f-rb2xm',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'pytorch-multi-io',
      'status': 'Running',
      'version': '55ff6626-d8b0-450c-aae3-520d9cc61034'}]},
   'model_statuses': {'models': [{'name': 'pytorch-multi-io',
      'sha': '792db9ee9f41aded3c1d4705f50ccdedd21cafb8b6232c03e4a849b6da1050a8',
      'status': 'Running',
      'version': '4ecb9de8-60fa-4a5f-b8fb-17e91a788cdb'}]}}],
 'engine_lbs': [{'ip': '10.4.2.29',
   'name': 'engine-lb-75cf576f7f-9v8db',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [10]:
mock_inference_data = [np.random.rand(10, 10), np.random.rand(10, 5)]
mock_dataframe = pd.DataFrame(
    {
        "input_1": mock_inference_data[0].tolist(),
        "input_2": mock_inference_data[1].tolist(),
    }
)

In [11]:
pipeline.infer(mock_dataframe)

,time,in.input_1,in.input_2,out.output_1,out.output_2,anomaly.count
0,2024-07-22 21:36:43.909,"[0.4349687559, 0.8157732994, 0.34490693, 0.831...","[0.7337659814, 0.7952291153, 0.5069130442, 0.2...","[-0.0009111166, 0.023788009, 0.1623653]","[-0.014985219, -0.14019005]",0
1,2024-07-22 21:36:43.909,"[0.1393807912, 0.2916602648, 0.8103438346, 0.5...","[0.8820245819, 0.9154249981, 0.1326196099, 0.5...","[-0.021424122, -0.09489694, 0.14667019]","[0.04212063, -0.032040358]",0
2,2024-07-22 21:36:43.909,"[0.9255647446, 0.7276030986, 0.3830442056, 0.1...","[0.6349132325, 0.3284397489, 0.277728319, 0.95...","[-0.028432503, -0.014432505, 0.29502067]","[0.021517769, -0.0652871]",0
3,2024-07-22 21:36:43.909,"[0.7482525537, 0.2073465906, 0.3910839444, 0.4...","[0.5981733, 0.2611876278, 0.0135134785, 0.3851...","[-0.015458949, -0.045656744, 0.21087806]","[-0.092412084, 0.08761616]",0
4,2024-07-22 21:36:43.909,"[0.2962533954, 0.5090249748, 0.0504361061, 0.3...","[0.1918274692, 0.0266545636, 0.0787328427, 0.6...","[0.0032953173, -0.005284533, 0.40087312]","[0.13697371, 0.009993628]",0
5,2024-07-22 21:36:43.909,"[0.0468508487, 0.853306844, 0.9646849501, 0.47...","[0.2819111799, 0.9529306281, 0.4387198424, 0.7...","[-0.015421592, -0.083104715, 0.33109748]","[0.21762016, -0.028640464]",0
6,2024-07-22 21:36:43.909,"[0.4154332023, 0.1384198555, 0.6395670577, 0.5...","[0.6599704935, 0.3999500169, 0.8010715884, 0.2...","[-0.03228599, 0.04157563, 0.2568084]","[-0.0081574805, -0.04890129]",0
7,2024-07-22 21:36:43.909,"[0.0180393032, 0.5956887035, 0.8733628886, 0.7...","[0.35872276, 0.0942387226, 0.1948172092, 0.958...","[-0.019901402, 0.0055049695, 0.3083496]","[0.044332013, 0.050565645]",0
8,2024-07-22 21:36:43.909,"[0.5503204754, 0.3348676322, 0.1425402513, 0.8...","[0.6779716785, 0.7648531854, 0.1105164503, 0.9...","[-0.0340152, -0.16801536, 0.2432538]","[-0.06369434, -0.01527518]",0
9,2024-07-22 21:36:43.909,"[0.2623454178, 0.633264752, 0.4281878886, 0.00...","[0.4273966523, 0.9877863343, 0.3566978016, 0.0...","[-0.10863065, 0.07400221, 0.03874488]","[0.13745771, -0.23087949]",0


### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [12]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok


name,pytorch-multi-io
created,2024-07-22 21:15:52.734458+00:00
last_updated,2024-07-22 21:36:31.107033+00:00
deployed,False
workspace_id,157
workspace_name,pytorch-multi-io
arch,x86
accel,none
tags,
versions,"55ff6626-d8b0-450c-aae3-520d9cc61034, 60733cb4-be1f-4bca-b0e6-76c575a2e90f, a5d4da15-80a4-4e00-8f6c-349de38ffc88"
steps,pytorch-multi-io
